<a href="https://colab.research.google.com/github/Kelnito/legendary-barnacle/blob/main/Lab_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas, warnings

warnings.filterwarnings("ignore")

In [2]:
origin = pandas.read_table("movies.csv", sep=",")

origin.iloc[:5]

,MovieID,Title,Genre
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [3]:
origin.tail()

,MovieID,Title,Genre
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation
9741,193609,Andrew Dice Clay: Dice Rules (1991),Comedy


In [4]:
origin.isnull().sum().sum()

0

In [5]:
origin["Genre"] = origin["Genre"].apply(lambda i : " ".join(i.split("|")))

origin.iloc[:5]

,MovieID,Title,Genre
0,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy
1,2,Jumanji (1995),Adventure Children Fantasy
2,3,Grumpier Old Men (1995),Comedy Romance
3,4,Waiting to Exhale (1995),Comedy Drama Romance
4,5,Father of the Bride Part II (1995),Comedy


in Our Goal Using `TfidfVectorizer`, Replacing `Sci-Fi` to `SciFi`, `Film-Noir` to `Filnoir`

In [6]:
replacer = {"Sci-Fi":"SciFi", "Film-Noir":"Filnoir"}

for i, t in replacer.items():

  origin["Genre"] = origin["Genre"].str.replace(i, t)

origin.iloc[:5]

,MovieID,Title,Genre
0,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy
1,2,Jumanji (1995),Adventure Children Fantasy
2,3,Grumpier Old Men (1995),Comedy Romance
3,4,Waiting to Exhale (1995),Comedy Drama Romance
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
nolist = "(no genres listed)"

origin = origin[origin["Genre"] != nolist]

origin.iloc[:5]

,MovieID,Title,Genre
0,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy
1,2,Jumanji (1995),Adventure Children Fantasy
2,3,Grumpier Old Men (1995),Comedy Romance
3,4,Waiting to Exhale (1995),Comedy Drama Romance
4,5,Father of the Bride Part II (1995),Comedy


In [8]:
Genres = []

for insect in origin["Genre"].values:

  insect = insect.split(" ")

  for lst in insect:

    if lst not in Genres: Genres.append(lst)

len(Genres)

19

Overall, Movie Lens Small Datasets Have 19 Unique Genres

In [9]:
Genres[:5]

['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy']

`re.search(r'\((\d{4})\)', title)`

In [10]:
import re

def Yearly(title):
  result = re.search(r'\((\d{4})\)', title)
  if result:
    return int(result.group(1))
  else:
    return 0

origin["Year"] = origin["Title"].apply(Yearly)

origin[origin["Year"] == origin["Year"].min()]

,MovieID,Title,Genre,Year
6059,40697,Babylon 5,SciFi,0
9031,140956,Ready Player One,Action SciFi Thriller,0
9179,149334,Nocturnal Animals,Drama Thriller,0
9367,162414,Moonlight,Drama,0


In [11]:
indelist = origin[origin["Year"] == 0].index.values

replacer = [2002, 2018, 2016, 2016]

for i in range(len(indelist)):

  origin.loc[indelist[i], "Year"] = replacer[i]

origin[origin.index.isin(indelist)]

,MovieID,Title,Genre,Year
6059,40697,Babylon 5,SciFi,2002
9031,140956,Ready Player One,Action SciFi Thriller,2018
9179,149334,Nocturnal Animals,Drama Thriller,2016
9367,162414,Moonlight,Drama,2016


In [12]:
origin.iloc[:5]

,MovieID,Title,Genre,Year
0,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy,1995
1,2,Jumanji (1995),Adventure Children Fantasy,1995
2,3,Grumpier Old Men (1995),Comedy Romance,1995
3,4,Waiting to Exhale (1995),Comedy Drama Romance,1995
4,5,Father of the Bride Part II (1995),Comedy,1995


Converting Genres Data to TF-IDF (Term Frequency-Inverse Document Frequency) Matrix

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words="english")

metrics = tfidf.fit_transform(origin["Genre"])

metrics.shape

(9708, 19)

In [14]:
tfidf.get_feature_names_out().tolist()[:5]

['action', 'adventure', 'animation', 'children', 'comedy']

In [15]:
len(tfidf.get_feature_names_out().tolist())

19

In [16]:
assert len(tfidf.get_feature_names_out().tolist()) == len(Genres), "Fail !"

In [17]:
len(metrics.todense())

9708

In [18]:
metrics.todense()[0]

matrix([[0.        , 0.4168036 , 0.51628768, 0.50489573, 0.26738778,
         0.        , 0.        , 0.        , 0.48301747, 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        ]])

In [19]:
metrics.todense()[1]

matrix([[0.        , 0.51229321, 0.        , 0.62056723, 0.        ,
         0.        , 0.        , 0.        , 0.59367667, 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        ]])

In [20]:
metrics.todense()[2]

matrix([[0.        , 0.        , 0.        , 0.        , 0.57070525,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.82115499,
         0.        , 0.        , 0.        , 0.        ]])

Perform Cosine Similarity to Measure Genres Similarity One Movie to Another

In [21]:
from sklearn.metrics.pairwise import cosine_similarity

similarities = cosine_similarity(metrics)

similarities.shape

(9708, 9708)

In [22]:
len(similarities[0])

9708

In [23]:
similarities[0][:10]

array([1.        , 0.8136036 , 0.15259961, 0.13500041, 0.26738778,
       0.        , 0.15259961, 0.65470981, 0.        , 0.26241348])

Our Movies, Jumanji, Have 81 % Similarity on Genres to Toy Story

In [24]:
similarities[1][:10]

array([0.8136036 , 1.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.80470368, 0.        , 0.32253235])

In [25]:
intler = lambda i : origin.iloc[i, 1]

titint = lambda t : origin[origin["Title"] == t].index.values[0]

In [26]:
t = titint("Jumanji (1995)")

t

1

In [27]:
moret = enumerate(similarities[int(t)])

molist = list(moret)

len(molist)

9708

In [28]:
molist[:5]

[(0, 0.8136035975025231), (1, 1.0), (2, 0.0), (3, 0.0), (4, 0.0)]